# Day 24

It's another simulation / path game.
Cycle detection of blizzards - cycles every w*h. So track places we've already been in the cycle - if we've already been there, we've lost to the version of ourselves that went before.
Can track all realities in a set. This will help collapse paths that end up in same place at same time.

Coord system: Row, Column
Strip the walls

In [11]:
testData = """#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#"""

from collections import defaultdict

directionVectors = {'>':(0,1),'<':(0,-1),'^':(-1,0),'v':(1,0)}

class Game:
    def __init__(self, input:str):
        self.time = 0
        self.trousersOftime = set() #where we store all our possible positions
        self.blizzards = defaultdict(lambda:[]) #dict of blizzard position and list of direction vector. 
        self.beenherebefore = set() #cycle detection
        for r, row in enumerate(input.splitlines()):
            if r == 0:
                #first line, detect start position
                self.width = len(row)-2
                for c, char in enumerate(row):
                    if char == '.':
                        self.startPos = (-1,c-1)
                        self.trousersOftime.add(self.startPos)
                        break
            elif row[1] == '#':
                #last row (making an assumption exit portal isn't in first possible possition)
                self.height = r-1
                for c, char in enumerate(row):
                    if char == '.':
                        self.endPos = (r-1,c-1)
                        break
            else:
                #process blizzards
                for c, char in enumerate(row):
                    if char in directionVectors:
                        pos = (r-1,c-1)
                        self.blizzards[pos].append(directionVectors[char])
        self.cycleTime = self.width * self.height
    
    def haveIBeenHereBefore(self, pos:tuple[int,int])->bool:
        key = (self.time % self.cycleTime) * 100000 + pos[0] * 1000 + pos[1]
        if key in self.beenherebefore:
            return True
        else:
            self.beenherebefore.add(key)
            return False
    
    def moveBlizzards(self):
        newBlizzards = defaultdict(lambda:[])
        for b, dir in self.blizzards.items():
            for d in dir:
                pos = (b[0]+d[0],b[1]+d[1])
                #world wrap
                if pos[0] < 0:
                    pos = (self.height - 1, pos[1])
                elif pos[0] >= self.height:
                    pos = (0, pos[1])
                elif pos[1] < 0:
                    pos = (pos[0],self.width - 1)
                elif pos[1] >= self.width:
                    pos = (pos[0],0)
                newBlizzards[pos].append(d)
        self.blizzards = newBlizzards


    def tick(self)->bool:
        #progress simulation one clock tick. Return True if simulation should contnue (i.e. False when simulation complete - goal met)
        continueSimulation = True
        self.time += 1
        #move all the blizzards
        self.moveBlizzards()
        #try all possible positions
        newTrousers = set()
        for pos in self.trousersOftime:
            newTrousers.add(pos) #stay still
            for d in directionVectors.values():
                newTrousers.add((pos[0]+d[0],pos[1]+d[1]))
        #remove collisions and out of bounds
        self.trousersOftime = newTrousers.copy()
        for pos in newTrousers:
            if pos == self.endPos:
                #GOAL
                return False
            else:
                if (((pos[0] < 0 or pos[0] >= self.height or pos[1] < 0 or pos[1] >= self.width) and pos!=self.startPos)
                    or pos in self.blizzards.keys()
                    or self.haveIBeenHereBefore(pos) ):
                    self.trousersOftime.remove(pos)

        return continueSimulation
    
    def run(self):
        while self.tick():
            if len(self.trousersOftime) == 0:
                raise Exception('Could not achieve goal. t='+str(self.time))
            if self.time % 10 == 0:
                print('Progress.... t='+str(self.time)+' trouser size='+str(len(self.trousersOftime)))
            continue
        print('Goal reached at t=: ' + str(self.time))

#tests
tg = Game(testData)
tg.run()





Progress.... t=10 trouser size=3
Goal reached at t=: 18


In [12]:
puzzleInput = open('day24input.txt').read()
g = Game(puzzleInput)
g.run()

Progress.... t=10 trouser size=10
Progress.... t=20 trouser size=39
Progress.... t=30 trouser size=88
Progress.... t=40 trouser size=143
Progress.... t=50 trouser size=211
Progress.... t=60 trouser size=279
Progress.... t=70 trouser size=308
Progress.... t=80 trouser size=372
Progress.... t=90 trouser size=397
Progress.... t=100 trouser size=381
Progress.... t=110 trouser size=495
Progress.... t=120 trouser size=519
Progress.... t=130 trouser size=589
Progress.... t=140 trouser size=591
Progress.... t=150 trouser size=596
Progress.... t=160 trouser size=689
Progress.... t=170 trouser size=756
Progress.... t=180 trouser size=780
Progress.... t=190 trouser size=854
Progress.... t=200 trouser size=795
Progress.... t=210 trouser size=902
Progress.... t=220 trouser size=1032
Goal reached at t=: 230
